In [262]:
import pandas as pd
import numpy as np
import pytz, sys,os
import plotly.express as px

from datetime import datetime as dt
from datetime import timezone
from dateutil import tz

sys.path.append(os.getcwd())


In [263]:

from PF_API_Shared import get_all_plants, get_plant_devices, Dispatch,TimeInterval, get_plants_metadata
from config import SUBSCRIPTION_KEY, CUSTOMER_ID


In [265]:

plant = 'Latitude'
start = dt(2022,8,1)
end = dt(2022,9,1)


In [266]:
dev_tags = [("Combiner", ["DC_CURRENT"]),
            ("Inverter", ["DC_CURRENT"]),
            ("Sensor Pyranometer POA", ["IRRADIANCE_POA"]),
                ("Meter",["AC_POWER"])]
  
rq_tags = []

plants = get_all_plants(SUBSCRIPTION_KEY, CUSTOMER_ID)  
plant_id = plants["id"][plants["name"] == plant].values[0]
dispatch = Dispatch(SUBSCRIPTION_KEY, '2096')
devices = get_plant_devices(SUBSCRIPTION_KEY,CUSTOMER_ID,plant_id)


In [267]:
for dtype, tags in dev_tags:
    dtype_devices = devices[devices['type']==dtype]
    
    df = dispatch.fetch_data(dtype_devices['id'], 
                                tags, 
                                query_range=TimeInterval(starttime=start,endtime=end))
    rq_tags.append(df)
inv_data = pd.concat(rq_tags, axis=1)
data = inv_data.copy(deep=True)

In [268]:
ident = [id for site, type, id, name in data.columns]
data.columns = ident
data.tz_convert()

In [270]:
data = data.set_index([data.index])
data.index = pd.to_datetime(data.index)
data = data[~data.index.duplicated(keep='first')]
data.index = data.index.tz_convert("US/Central")

In [271]:
"""For Filtered data, the data is filtered by the following parameters
        a. POA > 100
        b. (Meter Power > 1.5 MW) & (Meter Power < 25 MW)
        c. Hour > 12 (For shading purposes)
        d. All CBX's < 0 = 0
"""

data.loc[:, "Meter_Power"] = data.filter(regex="MTR").median(axis=1)
data.loc[:, "POA"] = data.filter(regex="MET").median(axis=1)

curtailment_limit = 499999

new_df = data.between_time("7:00", "17:00")
new_df = data[(data.loc[:,"POA"] > 100)&(data.loc[:,"Meter_Power"]<curtailment_limit*0.99)]
new_df[new_df<=0.00] = 0
day_df = new_df.resample("1D").sum()

day_df


C:\Users\dpinales\AppData\Local\Temp\ipykernel_31572\4223233658.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\dpinales\AppData\Local\Temp\ipykernel_31572\4223233658.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,2096.05.008.BLK01.PAD01.INV01.CMB01,2096.05.008.BLK01.PAD01.INV01.CMB02,2096.05.008.BLK01.PAD01.INV01.CMB03,2096.05.008.BLK01.PAD01.INV01.CMB04,2096.05.008.BLK01.PAD01.INV01.CMB05,2096.05.008.BLK01.PAD01.INV01.CMB06,2096.05.008.BLK01.PAD01.INV01.CMB07,2096.05.008.BLK01.PAD01.INV01.CMB08,2096.05.008.BLK01.PAD01.INV01.CMB09,2096.05.008.BLK01.PAD01.INV01.CMB10,...,2096.05.008.BLK01.PAD03.INV01,2096.05.008.BLK01.PAD04.INV01,2096.05.008.BLK01.PAD05.INV01,2096.05.008.MET07.PYR01,2096.05.008.MET08.PYR01,2096.05.008.MET09.PYR01,2096.05.008.METREF01.PYR01,2096.05.008.SWG01.MTR01,Meter_Power,POA
Timestamp,,,,,,,,,,,,,,,,,,,,,
2022-08-01 00:00:00-05:00,13993.069258,14025.299640,14480.775966,12421.553375,15520.198736,13436.997936,16371.643261,16438.931180,14984.963215,14330.350058,...,169746.313161,187295.187127,0.000000,0.0,0.0,0.0,79015.966576,7.715051e+05,7.715051e+05,79015.966576
2022-08-02 00:00:00-05:00,17991.496213,18121.189893,18644.509028,15960.191944,19932.820461,17333.105154,21225.778515,21450.828268,19314.246256,18849.100593,...,171388.270818,249149.173873,88058.911742,0.0,0.0,0.0,116113.328779,1.056993e+06,1.056993e+06,116113.328779
2022-08-03 00:00:00-05:00,21410.703954,21688.514238,22147.975403,18964.779773,23658.984455,20573.954283,25080.944344,25157.861684,22236.352315,21978.702430,...,154179.135834,210962.778365,357825.820079,0.0,0.0,0.0,123943.269709,1.476279e+06,1.476279e+06,123943.269709
2022-08-04 00:00:00-05:00,10052.946019,10273.715549,10327.041537,8845.399508,11046.125695,9560.116192,11596.690579,11727.360843,10301.529974,10253.350150,...,184360.647907,174606.774247,0.000000,0.0,0.0,0.0,86675.629784,5.551378e+05,5.551378e+05,86675.629784
2022-08-05 00:00:00-05:00,2348.118477,2520.397347,2480.488575,2136.599655,2663.889622,2318.546898,2783.512234,2778.501144,2786.780614,2391.913067,...,44481.347765,49687.656852,0.000000,0.0,0.0,0.0,79471.943560,1.297959e+05,1.297959e+05,79471.943560
2022-08-06 00:00:00-05:00,16895.890710,17149.638357,17535.428014,15007.272410,18776.201575,16354.463526,20062.840245,20175.411667,17769.192968,17652.261972,...,81567.241427,124412.861071,0.000000,0.0,0.0,0.0,126174.561150,9.063823e+05,9.063823e+05,126174.561150
2022-08-07 00:00:00-05:00,19696.243442,19790.366876,20451.727216,17480.917098,21801.228585,18906.665334,23029.571741,23035.252039,20256.354569,20200.872637,...,127691.275675,165783.777007,0.000000,0.0,0.0,0.0,122105.741308,1.050869e+06,1.050869e+06,122105.741308
2022-08-08 00:00:00-05:00,12009.622121,12129.849512,12501.234245,10759.839497,13439.931334,11785.332141,14328.113517,14610.444190,12915.770664,12705.807608,...,155788.472223,157392.201185,0.000000,0.0,0.0,0.0,122052.380604,6.855474e+05,6.855474e+05,122052.380604
2022-08-09 00:00:00-05:00,4694.285703,4689.644570,4823.959131,4119.433614,5149.515718,4458.558003,5429.936171,5414.142316,4727.299572,4593.630881,...,26894.136985,32655.898414,0.000000,0.0,0.0,0.0,61018.828009,2.396036e+05,2.396036e+05,61018.828009


In [272]:
# dc_issues = pd.read_excel("C:\\Users\\dpinales\\Desktop\\Energy_Opt\\data\\Gulf DC Issues.xlsx")
dc_meta = pd.read_excel("C:\\Users\\dpinales\\Desktop\\Energy_Opt\\data\\Project_Metadata_tables.xlsx", sheet_name="Combiner_table").filter(items= ['plant_name', 'string_count','DC Rating Power kW', 'Module IMP'], axis=1)


In [273]:
day_df = day_df.filter(regex="CMB")

dc_issues = dc_meta[dc_meta["plant_name"]==plant]  
dc_rating = [i for i in dc_issues['DC Rating Power kW'].values]

relative_dc = []

max_dc = max(dc_rating)

for i in dc_rating:
    dc = (i/max_dc)
    relative_dc.append(dc)

string_count = [s for s in dc_issues['string_count']]

cmb_relative = day_df.div(relative_dc)

cmb_relative.loc[:,'norm'] = cmb_relative.apply(pd.Series.nlargest, axis = 1, n = 5).median(axis = 1)
t = cmb_relative.div(cmb_relative['norm'], axis = 0).clip(lower = 0, upper = 1)
t = t.iloc[:,:-1]

down_strings = (string_count*(1-t)).round()

down_strings.to_csv('latitude_DC.csv')


In [260]:
e_downstrings = sum(int(down_strings[:]))
total_string = sum(int(string_count))
ratio_strings = e_downstrings/total_string

print(f"The estimated plants DC outage: {ratio_strings}")

TypeError: int() argument must be a string, a bytes-like object or a number, not 'DataFrame'

In [38]:
'''Summation of CMB/INV'''

inv = []
inv_sum = []
y = []

for x in new_df.columns:
    if "INV" in x and "CMB" not in x:
        inv.append(x)
        inv_sum.append(new_df[x].sum())

inv_df = pd.DataFrame(inv_sum, index=[inv], columns=['Sum_of_inv'])
inv_df = inv_df.loc[~(inv_df==0).all(axis=1)]
inv_df

,Sum_of_inv
2096.03.005.BLK01.PAD01.INV01,79244.879152
2096.03.005.BLK01.PAD01.INV02,69381.540895
2096.03.005.BLK01.PAD02.INV01,80810.328796
2096.03.005.BLK01.PAD02.INV02,77728.734407
2096.03.005.BLK01.PAD03.INV01,64783.282832
2096.03.005.BLK01.PAD03.INV02,79970.106391


In [ ]:
sum_of_cbx = cbx_data["Sum_of_CBX"]
sum_of_inv = inv_df["Sum_of_inv"]

def cmb_summation(a,b):
    total = (a/b)
    return total


In [50]:
# cbx_data.to_csv(f"{plant} - August_Daily_Derates.csv")
cbx_data.to_csv(f"{plant} - API Data.csv")